# Esse é um pequeno projeto de Limpeza de Dados 
#### Está simulando a importação de uma base de dados de um banco com o resultado do pagamento ou não de empréstimo

In [1]:
# Importando Biblioteca
import pandas as pd

In [6]:
base = pd.read_excel("ChavesClientes.xlsx",sheet_name = "base")

In [7]:
base.head()

,ID,ChaveSituacao,ClassRisco,CatCliente,Pagamento
0,1,32FC,Ccinza,Basic-Alpha,1
1,2,25MV,AAmarelo,Black,1
2,3,27MV,B-Amarelo,Basic-Beta,1
3,4,26FD,BAmarelo,Black,0
4,5,26FD,C-Amarelo,Black,0


#### Verificando Cardinalidade

In [9]:
base.groupby(["Pagamento", "ChaveSituacao"])["Pagamento"].count()

Pagamento  ChaveSituacao
0          26FD             2
           28FC             2
           28MD             1
           30FC             1
           31MD             1
1          25FD             1
           25FV             1
           25MV             1
           26MC             2
           27MC             1
           27MD             2
           27MV             1
           28FS             1
           29MV             1
           31MV             1
           32FC             1
Name: Pagamento, dtype: int64


# informação  do que representa cada coluna:
 
 ### ChaveSituacao: formado por:
        - Idade do cliente (a idade mínima para ser cliente é 18 anos)
        - Gênero do cliente:
            - M: Masculino
            - F: Feminino
        - Estado civil do cliente:
            - S: solteiro
            - D: divorciado
            - V: viúvo
   ### ClassRisco: formado por:
         - Classificação do cliente como (A,B,C) e indicador (+,- ou vazio)
         - Cor do cliente de acordo com um modelo de churn interno da empresa
   ### CatCliente: formado por:
        - Categoria do cartão: qual o tipo de cartão do cliente:
            - Basic
            - Black
            - Platinum
  ###  Categoria VIP: categoria do cliente VIP (caso exista)
            - Alpha
            - Beta

In [15]:
texto = '32FC'

In [16]:
texto[-1]

'C'

# Separando as Colunas ChaveSituacao

In [20]:
base["Idade"] = base.ChaveSituacao.str[:2]

In [19]:
base.head()

,ID,ChaveSituacao,ClassRisco,CatCliente,Pagamento,Idade
0,1,32FC,Ccinza,Basic-Alpha,1,32
1,2,25MV,AAmarelo,Black,1,25
2,3,27MV,B-Amarelo,Basic-Beta,1,27
3,4,26FD,BAmarelo,Black,0,26
4,5,26FD,C-Amarelo,Black,0,26


In [34]:
base["Genero"] = base.ChaveSituacao.str[2:3]
base["EstadoCivil"] = base.ChaveSituacao.str[-1]  # Certifique-se de usar sempre "EstadoCivil", sem espaço!


In [36]:
base = base.drop(columns=["Estado Civil"])

In [37]:
base.head()

,ID,ChaveSituacao,ClassRisco,CatCliente,Pagamento,Idade,Genero,EstadoCivil
0,1,32FC,Ccinza,Basic-Alpha,1,32,F,C
1,2,25MV,AAmarelo,Black,1,25,M,V
2,3,27MV,B-Amarelo,Basic-Beta,1,27,M,V
3,4,26FD,BAmarelo,Black,0,26,F,D
4,5,26FD,C-Amarelo,Black,0,26,F,D


In [38]:
base.groupby(["Pagamento", "EstadoCivil"])["Pagamento"].count()

Pagamento  EstadoCivil
0          C              3
           D              4
1          C              4
           D              3
           S              1
           V              5
Name: Pagamento, dtype: int64

In [39]:
texto = "Basic-Alpha"

In [40]:
texto.split("-")

['Basic', 'Alpha']

# Separando as Coluna CatCliente
#### Está sendo usado o '-' para separar, podendo assim criar novas colunas 
#### O get 0 está sendo usado para pegar o primeiro elemento 
#### Isso NaN significa que é um valor vazio



In [51]:
base["Categoria"] = base.CatCliente.str.split("-").str.get(0)
base["CatVIp"] = base.CatCliente.str.split("-").str.get(1)

In [54]:
base.head()

,ID,ChaveSituacao,ClassRisco,CatCliente,Pagamento,Idade,Genero,EstadoCivil,Categoria,CatVIp
0,1,32FC,Ccinza,Basic-Alpha,1,32,F,C,Basic,Alpha
1,2,25MV,AAmarelo,Black,1,25,M,V,Black,NaN
2,3,27MV,B-Amarelo,Basic-Beta,1,27,M,V,Basic,Beta
3,4,26FD,BAmarelo,Black,0,26,F,D,Black,NaN
4,5,26FD,C-Amarelo,Black,0,26,F,D,Black,NaN


In [53]:
base = base.drop(columns=["Cliente"])

# Separando as Coluna ClassRisco	
#### Importando a Biblioteca Regex, para conseguir separar melhor 
#### Nesse caso não será criado uma classe para essas cores, pois foi um erro 


In [57]:
import re

In [62]:
re.findall('^[A-Z][^A-Za-z]?', 'AAmerelo')

['A']

In [63]:
base["Risco"] = base.ClassRisco.apply(lambda x:re.findall('^[A-Z][^A-Za-z]?', x)[0])

In [64]:
base.head()

,ID,ChaveSituacao,ClassRisco,CatCliente,Pagamento,Idade,Genero,EstadoCivil,Categoria,CatVIp,Risco
0,1,32FC,Ccinza,Basic-Alpha,1,32,F,C,Basic,Alpha,C
1,2,25MV,AAmarelo,Black,1,25,M,V,Black,NaN,A
2,3,27MV,B-Amarelo,Basic-Beta,1,27,M,V,Basic,Beta,B-
3,4,26FD,BAmarelo,Black,0,26,F,D,Black,NaN,B
4,5,26FD,C-Amarelo,Black,0,26,F,D,Black,NaN,C-


In [66]:
base.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 20 entries, 0 to 19
Data columns (total 11 columns):
 #   Column         Non-Null Count  Dtype 
---  ------         --------------  ----- 
 0   ID             20 non-null     int64 
 1   ChaveSituacao  20 non-null     object
 2   ClassRisco     20 non-null     object
 3   CatCliente     20 non-null     object
 4   Pagamento      20 non-null     int64 
 5   Idade          20 non-null     object
 6   Genero         20 non-null     object
 7   EstadoCivil    20 non-null     object
 8   Categoria      20 non-null     object
 9   CatVIp         8 non-null      object
 10  Risco          20 non-null     object
dtypes: int64(2), object(9)
memory usage: 1.8+ KB


# Tem um Erro na Coluna Idade, ela não pode ser um objeto


In [67]:
base["Idade"] = pd.to_numeric(base["Idade"])

In [68]:
base.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 20 entries, 0 to 19
Data columns (total 11 columns):
 #   Column         Non-Null Count  Dtype 
---  ------         --------------  ----- 
 0   ID             20 non-null     int64 
 1   ChaveSituacao  20 non-null     object
 2   ClassRisco     20 non-null     object
 3   CatCliente     20 non-null     object
 4   Pagamento      20 non-null     int64 
 5   Idade          20 non-null     int64 
 6   Genero         20 non-null     object
 7   EstadoCivil    20 non-null     object
 8   Categoria      20 non-null     object
 9   CatVIp         8 non-null      object
 10  Risco          20 non-null     object
dtypes: int64(3), object(8)
memory usage: 1.8+ KB


# Mudando todos as linhas q estão como NaN no CatVip

In [72]:
base.loc[base.CatVIp.isnull(),"CatVIp"] = "Commum"

In [73]:
base.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 20 entries, 0 to 19
Data columns (total 11 columns):
 #   Column         Non-Null Count  Dtype 
---  ------         --------------  ----- 
 0   ID             20 non-null     int64 
 1   ChaveSituacao  20 non-null     object
 2   ClassRisco     20 non-null     object
 3   CatCliente     20 non-null     object
 4   Pagamento      20 non-null     int64 
 5   Idade          20 non-null     int64 
 6   Genero         20 non-null     object
 7   EstadoCivil    20 non-null     object
 8   Categoria      20 non-null     object
 9   CatVIp         20 non-null     object
 10  Risco          20 non-null     object
dtypes: int64(3), object(8)
memory usage: 1.8+ KB


In [77]:
base.groupby(["Pagamento", "Risco"])["Pagamento"].count()

Pagamento  Risco
0          B        2
           C-       5
1          A        3
           A-       4
           B-       2
           C        3
           C+       1
Name: Pagamento, dtype: int64